In [ ]:
import os
import sys
import pandas as pd

sys.path.append("..")
from rrg._data import DEFAULT_LABELS

In [ ]:
split = "validate"

In [ ]:
def get_best_results(dataset, section, df_true, val_ids):
    results = dict()
    for emb in ["biovilt", "gloria", "resnet50"]:
        if emb == "resnet50":
            continue

        for cls in ["classifiers", "lr-hyperparam", "svm-hyperparam", "rf-hyperparam", "xgb-hyperparam"]:
            for ths in ["roc", "pr"]:
                pred_path = f"/opt/gpudata/labrag/{dataset}-{section}-{emb}-{cls}/pred_{ths}.csv"

                if dataset == "chexpertplus" and emb == "biovilt":
                    continue
                elif dataset == "mimic" and emb == "gloria":
                    continue

                if not os.path.exists(pred_path):
                    print(f"skipping {pred_path}")
                    continue

                df_pred = pd.read_csv(pred_path).set_index("study_id")
                idxs = sorted(list(set(df_pred.index) & set(val_ids)))
                bin_pred = df_pred.loc[idxs, DEFAULT_LABELS] == 1
                bin_true = df_true.loc[idxs, DEFAULT_LABELS] == 1
                results[pred_path] = ((bin_pred == bin_true).sum(axis=0) / len(idxs))
    results = pd.DataFrame().from_dict(results, orient="index")
    df = df_true.copy()
    for label, pred_path in results.idxmax().items():
        temp = pd.read_csv(pred_path)
        assert (temp["study_id"] == df.index).all()
        df[label] = temp[label].to_list()
    return df

In [ ]:
for dataset in ["chexpertplus", "mimic"]:
    for section in ["findings", "impression"]:
        if dataset == "chexpertplus":
            splits = pd.read_csv("/opt/gpudata/chexpertplus/split.csv")
        else: # mimic
            splits = pd.read_csv("/opt/gpudata/mimic-cxr/mimic-cxr-2.0.0-split.csv")

        df_true = pd.read_csv(f"/opt/gpudata/cxr-derived/{dataset}-{section}-labels.csv").set_index("study_id")

        val_ids = splits.loc[splits["split"] == split, "study_id"]

        print(f"----- {dataset}/{section} -----")
        df = get_best_results(dataset, section, df_true, val_ids)
        df.reset_index().to_csv(f"/opt/gpudata/labrag/{dataset}-{section}-best-pred.csv", index=False)